#### 온라인 선호지수 도출하는 함수

#### [달라지는 변수]
#### 1)sampling 개수 from customer segmetation : 10000명, 20000명
#### 2)distance 종류 : cosine, euclidean, correlation
#### 3)kmeans clustering 적용 or not

In [2]:
import numpy as np
import pandas as pd
import sklearn
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os
import pickle

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine, euclidean
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance

In [4]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def get_mse(pred,actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return mean_squared_error(pred, actual)
def get_rmse(pred,actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return sqrt(mean_squared_error(pred, actual))

# 1. 구매선호지수

## 1) 10000명 sampling

In [16]:
#데이터 불러오기 및 초기화
Purchase_Count = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/data2/Purchase_Count.csv")
Purchase_Count = Purchase_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})
clustered_10000 = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/output2/clustered_10000.csv", encoding = "UTF-8")
clustered_10000 = clustered_10000.drop(['Unnamed: 0'], axis = 1)
Purchase_Count = pd.merge(Purchase_Count, clustered_10000[['CLNT_ID']], how = 'inner', on ='CLNT_ID')
row_names = Purchase_Count['CLNT_ID'].as_matrix()
col_names = Purchase_Count.columns.values
col_names = col_names[1:]
PC = Purchase_Count.as_matrix()
PC = PC[:,1:]
Purchase_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
#sparsity구하기
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Purchase_Count, test_size = 0.33, random_state = 50)
print(PC_train.shape)
print(PC_test.shape)
PC_train = pd.DataFrame(PC_train)
PC_test = pd.DataFrame(PC_test)
#array 형태로 변환
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()

Sparsity: {:4.2f}% 13.162443018558125
(5029, 37)
(2477, 37)


## 2) 20000명 sampling

In [26]:
#데이터 불러오기 및 초기화
Purchase_Count = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/data2/Purchase_Count.csv")
Purchase_Count = Purchase_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})
clustered_10000 = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/output2/clustered_20000.csv", encoding = "UTF-8")
clustered_10000 = clustered_10000.drop(['Unnamed: 0'], axis = 1)
Purchase_Count = pd.merge(Purchase_Count, clustered_10000[['CLNT_ID']], how = 'inner', on ='CLNT_ID')
row_names = Purchase_Count['CLNT_ID'].as_matrix()
col_names = Purchase_Count.columns.values
col_names = col_names[1:]
PC = Purchase_Count.as_matrix()
PC = PC[:,1:]
Purchase_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
#sparsity구하기
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Purchase_Count, test_size = 0.33, random_state = 50)
print(PC_train.shape)
print(PC_test.shape)
PC_train = pd.DataFrame(PC_train)
PC_test = pd.DataFrame(PC_test)
#array 형태로 변환
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()

Sparsity: {:4.2f}% 14.349619166222743
(10265, 37)
(5057, 37)


### 구매선호지수 계산

In [20]:
###파라메터 = k랑 코사인 
def purchase_prediction(train_mat, test_mat, metric_, k, output = False, scaling = False):
    if scaling == True:
        train_mat = (train_mat.T/train_mat.sum(axis=1)).T
        test_mat = (test_mat.T/test_mat.sum(axis=1)).T
    
    dist_out = 1 - pairwise_distances(train_mat, metric= metric_)

    #without kmeans clustering
    user_pred = dist_out.dot(train_mat)/np.array([np.abs(dist_out).sum(axis = 1)]).T
    print('w/o kmeans, train RMSE:  ',get_rmse(user_pred, train_mat))
    print('w/o kmeans, testRMSE:  ',get_rmse(user_pred, test_mat))
    print('w/o kmeans, train stdev:  ',np.std(user_pred - train_mat))

    
    #with kmeans clustering
    neigh = NearestNeighbors(k, metric_)
    neigh.fit(train_mat)
    top_k_distances, top_k_users = neigh.kneighbors(train_mat, return_distance = True)
    user_pred_k = np.zeros(train_mat.shape)
    
    for i in range(PC_train.shape[0]):
        user_pred_k[i,:] = top_k_distances[i].T.dot(train_mat[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis = 0)]).T  
    online_pref = pd.DataFrame(user_pred_k)
    na_element_index = np.where(np.isnan(online_pref))
    na_row_index = np.unique(na_element_index[0])
    removed_CLNT_ID = PC_train.index.values[na_row_index]
    existed_CLNT_ID = np.delete(PC_train.index.values,na_row_index)
    online_pref2 = online_pref.dropna()
    user_pred_k2 = online_pref2.as_matrix()
    train_mat2 = train_mat
    exist_idx = np.delete([range(online_pref.shape[0])],na_row_index)
    train_mat2 = train_mat2[exist_idx,:]
    print('w/ kmeans, train:  ',get_rmse(user_pred_k2, train_mat2))
    print('w/ kmeans, test:  ',get_rmse(user_pred_k2, test_mat))
    print('w/ kmeans, train stdev:  ',np.std(user_pred_k2 - train_mat2))


    #kmeans 적용한 선호지수 matrix 생성
    if output == True :
        col_names = Purchase_Count.columns.values
        row_names = existed_CLNT_ID
        Online_Preference_Count = pd.DataFrame(user_pred_k2, columns = col_names, index = row_names)
        return Online_Preference_Count

- 10000명 experiments

In [21]:
#10000명 sampling한 경우, cosine distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "cosine", 5, True)

w/o kmeans, train RMSE:   9.28956839596843
w/o kmeans, testRMSE:   9.847761522688266
w/o kmeans, train stdev:   3.45869924342


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   5.351571593734643
w/ kmeans, test:   10.930232796349458
w/ kmeans, train stdev:   2.12794348819


In [22]:
#10000명 sampling한 경우, euclidean distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "euclidean", 5, True)

w/o kmeans, train RMSE:   13.5154434821945
w/o kmeans, testRMSE:   14.28976224339411
w/o kmeans, train stdev:   5.10558624956


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   5.351571593734643
w/ kmeans, test:   10.930232796349458
w/ kmeans, train stdev:   2.12794348819


In [23]:
#10000명 sampling한 경우, correlation 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "correlation", 5, True)

w/o kmeans, train RMSE:   9.241663296687049
w/o kmeans, testRMSE:   9.968715144383735
w/o kmeans, train stdev:   3.40259162641


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   5.351571593734643
w/ kmeans, test:   10.930232796349458
w/ kmeans, train stdev:   2.12794348819


In [24]:
##Scaling 적용할 경우
#10000명 sampling한 경우, cosine distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "cosine", 5, True, True)
opc = purchase_prediction(train_mat,test_mat, "euclidean", 5, True, True)
opc = purchase_prediction(train_mat,test_mat, "correlation", 5, True, True)

w/o kmeans, train RMSE:   0.1677252391526864
w/o kmeans, testRMSE:   0.2641165824386254
w/o kmeans, train stdev:   0.0649180844801


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   0.05215676032242864
w/ kmeans, test:   0.292767126202287
w/ kmeans, train stdev:   0.0233360136695
w/o kmeans, train RMSE:   0.20399615995628823
w/o kmeans, testRMSE:   0.288027043302705
w/o kmeans, train stdev:   0.077398111886
w/ kmeans, train:   0.05215676032242864
w/ kmeans, test:   0.292767126202287
w/ kmeans, train stdev:   0.0233360136695
w/o kmeans, train RMSE:   0.16975341065612612
w/o kmeans, testRMSE:   0.2750306214814878
w/o kmeans, train stdev:   0.0633432620587
w/ kmeans, train:   0.05215676032242864
w/ kmeans, test:   0.292767126202287
w/ kmeans, train stdev:   0.0233360136695


- 20000명 experiments

In [27]:
#20000명 sampling한 경우, cosine distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "cosine", 5, True) 

w/o kmeans, train RMSE:   20.200295397717085
w/o kmeans, testRMSE:   24.22994776465791
w/o kmeans, train stdev:   7.69822238368


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   9.619011073912185
w/ kmeans, test:   24.719141408369648
w/ kmeans, train stdev:   3.93621962853


In [28]:
#20000명 sampling한 경우, euclidean distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "euclidean", 5, True) 

w/o kmeans, train RMSE:   27.468059645884846
w/o kmeans, testRMSE:   29.671633713123644
w/o kmeans, train stdev:   22.8442640957


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   9.619011073912185
w/ kmeans, test:   24.719141408369648
w/ kmeans, train stdev:   3.93621962853


In [29]:
#20000명 sampling한 경우, correlation 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "correlation", 5, True)

w/o kmeans, train RMSE:   23.474500361145516
w/o kmeans, testRMSE:   24.332507635967072
w/o kmeans, train stdev:   8.90987124053


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   9.619011073912185
w/ kmeans, test:   24.719141408369648
w/ kmeans, train stdev:   3.93621962853


In [30]:
##Scaling 적용할 경우
#20000명 sampling한 경우, cosine distance 쓴 경우, k = 5
opc = purchase_prediction(train_mat,test_mat, "cosine", 5, True, True)
opc = purchase_prediction(train_mat,test_mat, "euclidean", 5, True, True)
opc = purchase_prediction(train_mat,test_mat, "correlation", 5, True, True)

w/o kmeans, train RMSE:   0.16695596844688612
w/o kmeans, testRMSE:   0.2517848860790095
w/o kmeans, train stdev:   0.0670774424907


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   0.04635142046169629
w/ kmeans, test:   0.27879680468892437
w/ kmeans, train stdev:   0.021352061395
w/o kmeans, train RMSE:   0.21002514607048922
w/o kmeans, testRMSE:   0.27109046376027335
w/o kmeans, train stdev:   0.0823037325115
w/ kmeans, train:   0.04635142046169629
w/ kmeans, test:   0.27879680468892437
w/ kmeans, train stdev:   0.021352061395
w/o kmeans, train RMSE:   0.17371314563730972
w/o kmeans, testRMSE:   0.26374844476149106
w/o kmeans, train stdev:   0.0671125731231
w/ kmeans, train:   0.04635142046169629
w/ kmeans, test:   0.27879680468892437
w/ kmeans, train stdev:   0.021352061395


# 2. 검색선호지수

## 1)10000명 sampling

In [31]:
#데이터 불러오기 및 초기화
Search_Count = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/data2/Search_CNT.csv")
Search_Count = Search_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})
clustered_10000 = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/output2/clustered_10000.csv", encoding = "UTF-8")
clustered_10000 = clustered_10000.drop(['Unnamed: 0'], axis = 1)
Search_Count = pd.merge(Search_Count, clustered_10000[['CLNT_ID']], how = 'inner', on ='CLNT_ID')
row_names = Search_Count['CLNT_ID'].as_matrix()
col_names = Search_Count.columns.values
col_names = col_names[1:]
PC = Search_Count.as_matrix()
PC = PC[:,1:]
Search_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
#sparsity구하기
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Search_Count, test_size = 0.33, random_state = 50)
print(PC_train.shape)
print(PC_test.shape)
PC_train = pd.DataFrame(PC_train)
PC_test = pd.DataFrame(PC_test)
#array 형태로 변환
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()

Sparsity: {:4.2f}% 10.11659141155544
(5029, 37)
(2477, 37)


## 2)20000명 sampling

In [37]:
#데이터 불러오기 및 초기화
Search_Count = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/data2/Search_CNT.csv")
Search_Count = Search_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})
clustered_10000 = pd.read_csv("C:/Users/lsh93/Desktop/Lpoint/output2/clustered_20000.csv", encoding = "UTF-8")
clustered_10000 = clustered_10000.drop(['Unnamed: 0'], axis = 1)
Search_Count = pd.merge(Search_Count, clustered_10000[['CLNT_ID']], how = 'inner', on ='CLNT_ID')
row_names = Search_Count['CLNT_ID'].as_matrix()
col_names = Search_Count.columns.values
col_names = col_names[1:]
PC = Search_Count.as_matrix()
PC = PC[:,1:]
Search_Count = pd.DataFrame(PC, columns = col_names, index = row_names)
#sparsity구하기
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Search_Count, test_size = 0.33, random_state = 50)
print(PC_train.shape)
print(PC_test.shape)
PC_train = pd.DataFrame(PC_train)
PC_test = pd.DataFrame(PC_test)
#array 형태로 변환
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()

Sparsity: {:4.2f}% 10.898478428826948
(10265, 37)
(5057, 37)


### 검색선호지수 계산

In [32]:
###파라메터 = k랑 코사인 
def search_prediction(train_mat, test_mat, metric_, k, output = False, scaling = False):
    if scaling == True:
        train_mat = (train_mat.T/train_mat.sum(axis=1)).T
        test_mat = (test_mat.T/test_mat.sum(axis=1)).T
        
    dist_out = 1 - pairwise_distances(train_mat, metric= metric_)

    #without kmeans clustering
    user_pred = dist_out.dot(train_mat)/np.array([np.abs(dist_out).sum(axis = 1)]).T
    print('w/o kmeans, train:  ',get_rmse(user_pred, train_mat))
    print('w/o kmeans, test:  ',get_rmse(user_pred, test_mat))
    print('w/o kmeans, train stdev:  ',np.std(user_pred - train_mat))


    #with kmeans clustering
    neigh = NearestNeighbors(k, metric_)
    neigh.fit(train_mat)
    top_k_distances, top_k_users = neigh.kneighbors(train_mat, return_distance = True)
    user_pred_k = np.zeros(train_mat.shape)
    
    for i in range(PC_train.shape[0]):
        user_pred_k[i,:] = top_k_distances[i].T.dot(train_mat[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis = 0)]).T  
    online_pref = pd.DataFrame(user_pred_k)
    na_element_index = np.where(np.isnan(online_pref))
    na_row_index = np.unique(na_element_index[0])
    removed_CLNT_ID = PC_train.index.values[na_row_index]
    existed_CLNT_ID = np.delete(PC_train.index.values,na_row_index)
    online_pref2 = online_pref.dropna()
    user_pred_k2 = online_pref2.as_matrix()
    train_mat2 = train_mat
    exist_idx = np.delete([range(online_pref.shape[0])],na_row_index)
    train_mat2 = train_mat2[exist_idx,:]
    print('w/ kmeans, train:  ',get_rmse(user_pred_k2, train_mat2))
    print('w/ kmeans, test:  ',get_rmse(user_pred_k2, test_mat))
    print('w/ kmeans, train stdev:  ',np.std(user_pred_k2 - train_mat2))


    #kmeans 적용한 선호지수 matrix 생성
    if output == True :
        col_names = Search_Count.columns.values
        row_names = existed_CLNT_ID
        Online_Preference_Search = pd.DataFrame(user_pred_k2, columns = col_names, index = row_names)
        return Online_Preference_Search

- 10000명 experiments

In [33]:
#10000명 sampling한 경우, cosine distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "cosine", 5, True)

w/o kmeans, train:   67.94028345092731
w/o kmeans, test:   30.401526022998492
w/o kmeans, train stdev:   21.8246096179


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   54.874176894191265
w/ kmeans, test:   36.88540631592994
w/ kmeans, train stdev:   19.4037783917


In [34]:
#10000명 sampling한 경우, euclidean distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "euclidean", 5, True)

w/o kmeans, train:   127.77850213164407
w/o kmeans, test:   126.66256973971018
w/o kmeans, train stdev:   61.0784275376


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   54.874176894191265
w/ kmeans, test:   36.88540631592994
w/ kmeans, train stdev:   19.4037783917


In [35]:
#10000명 sampling한 경우, correlation 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "correlation", 5, True)

w/o kmeans, train:   67.81191797494421
w/o kmeans, test:   30.944524554951546
w/o kmeans, train stdev:   21.7199837565


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   54.874176894191265
w/ kmeans, test:   36.88540631592994
w/ kmeans, train stdev:   19.4037783917


In [36]:
##Scaling 적용할 경우
#10000명 sampling한 경우, cosine distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "cosine", 5, True, True)
ops = search_prediction(train_mat,test_mat, "euclidean", 5, True, True)
ops = search_prediction(train_mat,test_mat, "correlation", 5, True, True)

w/o kmeans, train:   0.18979565581422897
w/o kmeans, test:   0.35772623973582546
w/o kmeans, train stdev:   0.0638590724186


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   0.0576526871778342
w/ kmeans, test:   0.3793810443858214
w/ kmeans, train stdev:   0.0237134132352
w/o kmeans, train:   0.2777658169346724
w/o kmeans, test:   0.4042345534190704
w/o kmeans, train stdev:   0.0909833037868
w/ kmeans, train:   0.0576526871778342
w/ kmeans, test:   0.3793810443858214
w/ kmeans, train stdev:   0.0237134132352
w/o kmeans, train:   0.20530686716922966
w/o kmeans, test:   0.3728430723669624
w/o kmeans, train stdev:   0.0663741847972
w/ kmeans, train:   0.0576526871778342
w/ kmeans, test:   0.3793810443858214
w/ kmeans, train stdev:   0.0237134132352


- 20000명 experiments

In [38]:
#20000명 sampling한 경우, cosine distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "cosine", 5, True)

w/o kmeans, train:   51.0307569624951
w/o kmeans, test:   38.43620901476732
w/o kmeans, train stdev:   16.9885693726


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   39.7284091550407
w/ kmeans, test:   42.160758229311156
w/ kmeans, train stdev:   14.5334888121


In [39]:
#20000명 sampling한 경우, euclidean distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "euclidean", 5, True)

w/o kmeans, train:   81.6953140808291
w/o kmeans, test:   84.7552965030701
w/o kmeans, train stdev:   36.4554173034


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   39.7284091550407
w/ kmeans, test:   42.160758229311156
w/ kmeans, train stdev:   14.5334888121


In [40]:
#20000명 sampling한 경우, correlation 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "correlation", 5, True)

w/o kmeans, train:   50.92519083332362
w/o kmeans, test:   39.01708921288831
w/o kmeans, train stdev:   16.8726554545


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   39.7284091550407
w/ kmeans, test:   42.160758229311156
w/ kmeans, train stdev:   14.5334888121


In [41]:
##Scaling 적용할 경우
#20000명 sampling한 경우, cosine distance 쓴 경우, k = 5
ops = search_prediction(train_mat,test_mat, "cosine", 5, True, True)
ops = search_prediction(train_mat,test_mat, "euclidean", 5, True, True)
ops = search_prediction(train_mat,test_mat, "correlation", 5, True, True)

w/o kmeans, train:   0.18925953599710146
w/o kmeans, test:   0.3345583825825382
w/o kmeans, train stdev:   0.0656712683558


C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in true_divide


w/ kmeans, train:   0.0517999992664449
w/ kmeans, test:   0.35917052432961155
w/ kmeans, train stdev:   0.021880093754
w/o kmeans, train:   0.2797723972398393
w/o kmeans, test:   0.37251701299294415
w/o kmeans, train stdev:   0.093898895702
w/ kmeans, train:   0.0517999992664449
w/ kmeans, test:   0.35917052432961155
w/ kmeans, train stdev:   0.021880093754
w/o kmeans, train:   0.2107317645353789
w/o kmeans, test:   0.35000895265888904
w/o kmeans, train stdev:   0.0701529612105
w/ kmeans, train:   0.0517999992664449
w/ kmeans, test:   0.35917052432961155
w/ kmeans, train stdev:   0.021880093754
